In [36]:
import mysql.connector as connection
import pandas as pd
from mysql.connector import Error
from pyspark.sql import SparkSession
import pyspark

appName = "PySpark MySQL - parquet"
master = "local"

spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

# Base de datos
db_name = 'prueba'

# Ingresar contraseña
pw = '360Donaldo'

ciudades = ['CDMX', 'MTY', 'MER']

data = []

for ciudad in ciudades:
    try:
        mydb = connection.connect(host="localhost", database = db_name ,user="donaldo", passwd=pw ,use_pure=True)
        query = """SELECT Datos.ciudad_id, Datos.respuesta_id, Datos.temperatura_actual, Datos.humedad_relativa
                FROM Datos
                WHERE Datos.ciudad_id = {}
                ORDER BY Datos.respuesta_id;""".format('"'+ciudad+'"')
        pandas_df = pd.read_sql(query, mydb)
        mydb.close() #close the connection

    except Error as err:
        print(f"Error: '{err}' ")


    # Convert Pandas dataframe to spark DataFrame
    df = spark.createDataFrame(pandas_df)

    # %%
    variables = ["temperatura_actual", "humedad_relativa"]

    aux = 2

    data_ciudad = [[df.collect()[-1][0]]]

    for variable in variables:

        max_var = df.agg({variable: "max"}).collect()[0]
        var_max = max_var["max({})".format(variable)]

        min_var = df.agg({variable: "min"}).collect()[0]
        var_min = min_var["min({})".format(variable)]

        avg_var = df.agg({variable: "avg"}).collect()[0]
        var_avg = avg_var["avg({})".format(variable)]

        var_current = df.collect()[-1][aux]
        aux+=1

        data_ciudad.append([var_max, var_min, var_avg, var_current])

    data_ciudad.append([df.collect()[-1][1].split('-')[1]])
    data_ciudad = tuple([item for sublist in data_ciudad for item in sublist])

    print(data_ciudad)

    data.append(data_ciudad)


print(data)

/tmp/ipykernel_64664/3163312150.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pandas_df = pd.read_sql(query, mydb)


('CDMX', 26.0, 26.0, 26.0, 26.0, 36.5, 36.5, 36.5, 36.5, '140620232102')
('MTY', 36.0, 36.0, 36.0, 36.0, 37.0, 37.0, 37.0, 37.0, '140620232102')
('MER', 33.0, 33.0, 33.0, 33.0, 49.4, 49.4, 49.4, 49.4, '140620232102')
[('CDMX', 26.0, 26.0, 26.0, 26.0, 36.5, 36.5, 36.5, 36.5, '140620232102'), ('MTY', 36.0, 36.0, 36.0, 36.0, 37.0, 37.0, 37.0, 37.0, '140620232102'), ('MER', 33.0, 33.0, 33.0, 33.0, 49.4, 49.4, 49.4, 49.4, '140620232102')]


In [37]:
#spark=SparkSession.builder.appName("parquetFile").getOrCreate()
columns=["ciudad","temperatura_max","temperatura_min","temperatura_prom","temperatura_actual",
         "humedad_max","humedad_min","humedad_prom","humedad_actual", "corrida"]
df_parquet=spark.createDataFrame(data, columns)
df_parquet.write.mode("overwrite").parquet('./parquet/clima.parquet')

In [38]:
df_parquet.show()

+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+------------+
|ciudad|temperatura_max|temperatura_min|temperatura_prom|temperatura_actual|humedad_max|humedad_min|humedad_prom|humedad_actual|     corrida|
+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+------------+
|  CDMX|           26.0|           26.0|            26.0|              26.0|       36.5|       36.5|        36.5|          36.5|140620232102|
|   MTY|           36.0|           36.0|            36.0|              36.0|       37.0|       37.0|        37.0|          37.0|140620232102|
|   MER|           33.0|           33.0|            33.0|              33.0|       49.4|       49.4|        49.4|          49.4|140620232102|
+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+------------+



In [39]:
df_parquet.write.mode('append').parquet("./parquet/clima.parquet")

In [45]:
parDF=spark.read.parquet("./parquet/clima.parquet/corrida=140620232058")

In [46]:
parDF.show()

+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+
|ciudad|temperatura_max|temperatura_min|temperatura_prom|temperatura_actual|humedad_max|humedad_min|humedad_prom|humedad_actual|
+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+
|  CDMX|           26.0|           26.0|            26.0|              26.0|       36.5|       36.5|        36.5|          36.5|
|   MTY|           36.0|           36.0|            36.0|              36.0|       37.0|       37.0|        37.0|          37.0|
|   MER|           33.0|           33.0|            33.0|              33.0|       49.4|       49.4|        49.4|          49.4|
|  CDMX|           26.0|           26.0|            26.0|              26.0|       36.5|       36.5|        36.5|          36.5|
|   MTY|           36.0|           36.0|            36.0|              36.0|       37.0|       37

In [42]:

df_parquet.write.partitionBy("corrida").mode("append").parquet("./parquet/clima.parquet")

In [49]:
import os
not os.listdir('./parquet')

True

In [51]:
parDF=spark.read.parquet("./parquet/clima.parquet/corrida=140620232102")

parDF.show()

+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+
|ciudad|temperatura_max|temperatura_min|temperatura_prom|temperatura_actual|humedad_max|humedad_min|humedad_prom|humedad_actual|
+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+
|  CDMX|           26.0|           26.0|            26.0|              26.0|       36.5|       36.5|        36.5|          36.5|
|   MTY|           36.0|           36.0|            36.0|              36.0|       37.0|       37.0|        37.0|          37.0|
|   MER|           33.0|           33.0|            33.0|              33.0|       49.4|       49.4|        49.4|          49.4|
+------+---------------+---------------+----------------+------------------+-----------+-----------+------------+--------------+



In [53]:
parDF=spark.read.parquet("./parquet/clima.parquet")

parDF.show()

+------+---------------+---------------+------------------+------------------+-----------+-----------+------------------+--------------+------------+
|ciudad|temperatura_max|temperatura_min|  temperatura_prom|temperatura_actual|humedad_max|humedad_min|      humedad_prom|humedad_actual|     corrida|
+------+---------------+---------------+------------------+------------------+-----------+-----------+------------------+--------------+------------+
|  CDMX|           26.0|           25.0|25.714285714285715|              25.0|       38.8|       36.5| 37.15714285714286|          38.8|140620232149|
|   MTY|           36.0|           34.0| 35.42857142857143|              34.0|       38.8|       37.0| 37.51428571428572|          38.8|140620232149|
|   MER|           33.0|           32.0|32.714285714285715|              32.0|       52.3|       49.4|50.228571428571435|          52.3|140620232149|
|  CDMX|           26.0|           25.0|25.833333333333332|              25.0|       38.8|       36.